<a href="https://colab.research.google.com/github/pranjalkamboj/Fine-tuning-Lora-Qlora/blob/main/Copy_of_Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q accelerate peft bitsandbytes transformers trl


In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer


In [ ]:
!rm -rf /root/.cache/huggingface/datasets

In [ ]:
dataset = load_dataset("tatsu-lab/alpaca", )
dataset = dataset['train'].shuffle(seed=42).select(range(1000))


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/tatsu-lab___parquet/tatsu-lab--alpaca-2b32f0433506ef5f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
def convert_to_llama2_format(example):
    system_prompt = "You are a helpful assistant."

    instruction = example['instruction'].strip()
    input_text = example['input'].strip()
    output_text = example['output'].strip()

    if input_text == "":
        full_instruction = instruction
    else:
        full_instruction = f"{instruction}\n{input_text}"

    formatted_prompt = f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{full_instruction} [/INST] {output_text}</s>"

    return {"text": formatted_prompt}

# Apply the formatting
formatted_dataset = dataset.map(convert_to_llama2_format)
print(formatted_dataset[0]['text'])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

<s>[INST] <<SYS>>
You are a helpful assistant.
<</SYS>>

What would be the best type of exercise for a person who has arthritis? [/INST] For someone with arthritis, the best type of exercise would be low-impact activities like yoga, swimming, or walking. These exercises provide the benefits of exercise without exacerbating the symptoms of arthritis.</s>
